VWAP Trading Strategy

The strategy that revolves around the Volume Weighted Average Price (VWAP) involves utilizing this particular indicator as a dynamic level of support and resistance within the market. The underlying concept is straightforward yet effective: when the market price of an asset is trading below the VWAP, it is often interpreted as a potential buying opportunity, suggesting that the asset may be undervalued at that moment. Conversely, when the price is trading above the VWAP, it might indicate that the asset is overvalued, presenting a good opportunity to sell or initiate a short position.

To elaborate further, here’s a basic strategy that you can implement using the VWAP as a guiding tool:

Buy Signal: When the price of the asset crosses above the VWAP after having been below it for a period of time, this movement is indicative of a potential upward reversal in the market trend. Such a scenario could present an opportune moment to enter a long position, as it suggests that the buying momentum is gaining strength and the price may continue to rise.

Sell Signal: When the price of a security crosses below the Volume Weighted Average Price (VWAP) after having been above it for a period of time, this movement suggests a potential downward reversal in the market trend. Such a crossover can be interpreted as a bearish signal, indicating that the asset may be losing its upward momentum and could be poised.


Implementing the Strategy on S&P 500 Companies

To thoroughly evaluate the effectiveness of this strategy, I decided to implement it on the 50 largest companies within the S&P 500 index. This selection was made to ensure a diverse representation of industries and market capitalizations, providing a robust testing ground for the strategy. The core of this strategy involves utilizing historical intraday trading.

In [6]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
import openpyxl

# List of top 50 S&P 500 companies by market cap
top_50_sp500 = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'BRK-B', 'TSLA', 'NVDA', 'JPM', 'JNJ',
    'V', 'UNH', 'HD', 'PG', 'DIS', 'MA', 'PYPL', 'VZ', 'ADBE', 'NFLX',
    'INTC', 'CMCSA', 'KO', 'PFE', 'PEP', 'T', 'XOM', 'CSCO', 'ABT', 'MRK',
    'NKE', 'ABBV', 'CRM', 'AVGO', 'MCD', 'QCOM', 'TXN', 'ACN', 'MDT', 'COST',
    'NEE', 'DHR', 'WMT', 'AMGN', 'HON', 'IBM', 'GE', 'LOW', 'CAT', 'BA'
]

# Parameters
start_date = '2020-01-01'
end_date = '2023-01-01'
cash = 10000
commission = 0.002

# Function to calculate VWAP
def calculate_vwap(data):
    cum_volume = data['Volume'].cumsum()
    cum_vwap = (data['Close'] * data['Volume']).cumsum() / cum_volume
    return cum_vwap

# Define the VWAP Strategy
class VWAPStrategy(Strategy):
    def init(self):
        self.vwap = self.I(calculate_vwap, self.data.df)

    def next(self):
        if self.data.Close[-1] > self.vwap[-1] and self.data.Close[-2] <= self.vwap[-2]:
            self.buy()
        elif self.data.Close[-1] < self.vwap[-1] and self.data.Close[-2] >= self.vwap[-2]:
            self.sell()

# Function to run the backtest for a list of tickers
def run_backtests(tickers, start_date, end_date, cash, commission):
    all_metrics = []

    for ticker in tickers:
        try:
            # Fetch historical intraday data
            data = yf.download(ticker, start=start_date, end=end_date, interval="1d")

            if data.empty:
                print(f"No data for {ticker}. Skipping...")
                continue

            # Ensure there's enough data for the calculation
            if len(data) < 2:
                print(f"Not enough data for {ticker}. Skipping...")
                continue

            # Run the backtest
            bt = Backtest(data, VWAPStrategy, cash=cash, commission=commission)
            stats = bt.run()

            # Collect metrics
            metrics = {
                "Stock": ticker,
                "Start": stats['Start'],
                "End": stats['End'],
                "Duration": stats['Duration'],
                "Equity Final [$]": stats['Equity Final [$]'],
                "Equity Peak [$]": stats['Equity Peak [$]'],
                "Return [%]": stats['Return [%]'],
                "Buy & Hold Return [%]": stats['Buy & Hold Return [%]'],
                "Max. Drawdown [%]": stats['Max. Drawdown [%]'],
                "Avg. Drawdown [%]": stats['Avg. Drawdown [%]'],
                "Max. Drawdown Duration": stats['Max. Drawdown Duration'],
                "Trades": stats['# Trades'],
                "Win Rate [%]": stats['Win Rate [%]'],
                "Best Trade [%]": stats['Best Trade [%]'],
                "Worst Trade [%]": stats['Worst Trade [%]'],
                "Avg. Trade [%]": stats['Avg. Trade [%]'],
                "Max. Trade Duration": stats['Max. Trade Duration'],
                "Avg. Trade Duration": stats['Avg. Trade Duration'],
                "Profit Factor": stats['Profit Factor'],
                "Expectancy [%]": stats['Expectancy [%]'],
                "Sharpe Ratio": stats['Sharpe Ratio'],
                "Sortino Ratio": stats['Sortino Ratio'],
            }

            all_metrics.append(metrics)
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            continue

    # Convert to DataFrame
    metrics_df = pd.DataFrame(all_metrics)

    # Save to Excel
    metrics_df.to_excel("top_50_sp500_vwap_metrics.xlsx", index=False)

    return metrics_df

# Run the backtests
metrics_df = run_backtests(top_50_sp500, start_date, end_date, cash, commission)

# Print the results
print(metrics_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in scalar power
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
    Stock      Start        End  D

Brief Explanation of the Code

The provided code implements a comprehensive trading strategy that leverages the Volume Weighted Average Price (VWAP) indicator to make informed trading decisions. This strategy is designed to analyze and execute trades based on the VWAP, which is a popular technical analysis tool used by traders to assess the average price a security has traded at throughout the day, weighted by volume. Below is an expanded breakdown of the key components of this code:

1. Data Download: The code utilizes the yfinance library to efficiently download historical intraday trading data for the top 50 companies listed in the S&P 500 index. The data spans a period from January 1, 2020, to January 1, 2023. This extensive dataset provides a robust foundation for analyzing market trends and testing the trading strategy over a significant timeframe.

2. VWAP Calculation: The calculate_vwap function is a crucial part of the strategy, as it computes the VWAP by taking the cumulative sum of the product of the closing prices and the trading volume, and then dividing it by the cumulative sum of the volume. This calculation results in a price that is weighted by the volume, offering a more accurate reflection of where the majority of trading activity has occurred. The VWAP serves as a benchmark for evaluating the current price levels.

3. VWAP Strategy Implementation: The VWAPStrategy class encapsulates the core trading logic. Within the __init__ method, the VWAP is calculated for the entire dataset, setting the stage for the strategy's execution. The `next` method is where the strategy actively monitors the market for potential trading signals. It does so by checking for crossovers between the current market price and the VWAP. Specifically, if the current price crosses above the VWAP, the strategy interprets this as a bullish signal and generates a buy order. Conversely, if the price crosses below the VWAP,